# Satellite Change Detection System

This notebook demonstrates various techniques for detecting changes between before and after satellite images. It includes image differencing, thresholding, Change Vector Analysis (CVA), and deep learning approaches.

**Features:**
- Image preprocessing and alignment
- Multiple change detection methods
- Deep learning-based semantic change detection
- Visualization and statistics
- Export results as GeoTIFF or PNG

Works seamlessly in Google Colab!

## 1. Install Required Libraries

In [ ]:
import subprocess
import sys

# Install required packages
packages = [
    'opencv-python',
    'scikit-image',
    'rasterio',
    'geopandas',
    'tensorflow',
    'torch',
    'torchvision'
]

print("Installing required packages...")
for package in packages:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"✓ {package} installed")
    except:
        print(f"⚠ {package} installation skipped (may already be installed)")

print("\nAll packages installed successfully!")

## 2. Import Libraries and Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import requests
import os
from urllib.request import urlretrieve
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Check if running in Google Colab
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running in Google Colab")
except:
    IN_COLAB = False
    print("Running locally")

# Set up matplotlib
plt.style.use('default')
%matplotlib inline

## 3. Mount Google Drive (Optional)

In [ ]:
if IN_COLAB:
    drive.mount('/content/drive')
    working_dir = '/content/drive/MyDrive/satellite_change_detection'
    os.makedirs(working_dir, exist_ok=True)
    print(f"Google Drive mounted. Working directory: {working_dir}")
else:
    working_dir = './satellite_data'
    os.makedirs(working_dir, exist_ok=True)
    print(f"Working directory: {working_dir}")

## 4. Download Sample Satellite Images

In [ ]:
def create_sample_satellite_images(size=256):
    """
    Create sample satellite images for demonstration
    Simulates before and after satellite imagery
    """
    np.random.seed(42)
    
    # Create before image (base landscape)
    before = np.zeros((size, size, 3), dtype=np.uint8)
    
    # Add background (green landscape)
    before[:, :, 1] = np.random.randint(100, 150, (size, size))  # Green channel
    before[:, :, 0] = np.random.randint(60, 100, (size, size))   # Blue channel
    
    # Add some natural features
    cv2.circle(before, (50, 50), 20, (50, 100, 200), -1)  # Water body
    cv2.circle(before, (200, 200), 30, (180, 180, 180), -1)  # Building
    
    # Create after image (with changes)
    after = before.copy()
    
    # Simulate urban development (change)
    cv2.rectangle(after, (100, 100), (180, 180), (150, 150, 150), -1)  # New development
    cv2.circle(after, (50, 50), 20, (100, 150, 200), -1)  # Water expansion
    
    # Simulate forest loss (deforestation area)
    after[150:200, 150:200, :] = [100, 100, 100]
    
    return before, after

# Create sample images
print("Creating sample satellite images...")
before_img, after_img = create_sample_satellite_images(size=256)

print("Sample images created successfully!")
print(f"Before image shape: {before_img.shape}")
print(f"After image shape: {after_img.shape}")

## 5. Load and Visualize Before/After Images

In [ ]:
def visualize_images(before, after, title1="Before", title2="After"):
    """Visualize before and after images side by side"""
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].imshow(cv2.cvtColor(before, cv2.COLOR_BGR2RGB))
    axes[0].set_title(title1, fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(after, cv2.COLOR_BGR2RGB))
    axes[1].set_title(title2, fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize the images
visualize_images(before_img, after_img, "Before Image", "After Image")

## 6. Preprocess Images

In [ ]:
def preprocess_images(before, after, target_size=None):
    """
    Preprocess satellite images:
    - Resize to same dimensions
    - Normalize pixel values
    - Align images if needed
    """
    # Resize if needed
    if target_size:
        before = cv2.resize(before, target_size)
        after = cv2.resize(after, target_size)
    
    # Ensure same dimensions
    if before.shape != after.shape:
        h, w = min(before.shape[0], after.shape[0]), min(before.shape[1], after.shape[1])
        before = cv2.resize(before, (w, h))
        after = cv2.resize(after, (w, h))
    
    # Normalize (optional, helps with some methods)
    before_normalized = before.astype(np.float32) / 255.0
    after_normalized = after.astype(np.float32) / 255.0
    
    return before, after, before_normalized, after_normalized

# Preprocess images
before_proc, after_proc, before_norm, after_norm = preprocess_images(before_img, after_img)

print("Preprocessing complete!")
print(f"Processed image shape: {before_proc.shape}")
print(f"Normalized value range: [{before_norm.min():.2f}, {before_norm.max():.2f}]")

## 7. Image Differencing Method

In [ ]:
def image_differencing(before, after):
    """
    Simple image differencing method
    Computes absolute difference between images
    """
    # Convert to grayscale for easier analysis
    before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
    after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)
    
    # Compute absolute difference
    diff = cv2.absdiff(after_gray, before_gray)
    
    return diff

# Apply image differencing
diff_image = image_differencing(before_proc, after_proc)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(cv2.cvtColor(before_proc, cv2.COLOR_BGR2RGB))
axes[0].set_title('Before', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(after_proc, cv2.COLOR_BGR2RGB))
axes[1].set_title('After', fontsize=12, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(diff_image, cmap='hot')
axes[2].set_title('Difference Map', fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"Difference image range: [{diff_image.min()}, {diff_image.max()}]")

## 8. Thresholding for Change Detection

In [ ]:
def threshold_changes(diff_image, threshold=30, use_adaptive=False):
    """
    Apply thresholding to create binary change mask
    """
    if use_adaptive:
        # Adaptive thresholding
        change_mask = cv2.adaptiveThreshold(
            diff_image, 255, 
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
            cv2.THRESH_BINARY, 11, 2
        )
    else:
        # Simple thresholding
        _, change_mask = cv2.threshold(diff_image, threshold, 255, cv2.THRESH_BINARY)
    
    # Apply morphological operations to reduce noise
    kernel = np.ones((3, 3), np.uint8)
    change_mask = cv2.morphologyEx(change_mask, cv2.MORPH_OPEN, kernel)
    change_mask = cv2.morphologyEx(change_mask, cv2.MORPH_CLOSE, kernel)
    
    return change_mask

# Try different thresholds
thresholds = [20, 30, 40]
fig, axes = plt.subplots(1, len(thresholds) + 1, figsize=(16, 4))

axes[0].imshow(diff_image, cmap='hot')
axes[0].set_title('Difference Map', fontsize=11, fontweight='bold')
axes[0].axis('off')

for i, thresh in enumerate(thresholds):
    mask = threshold_changes(diff_image, threshold=thresh)
    axes[i+1].imshow(mask, cmap='gray')
    axes[i+1].set_title(f'Threshold = {thresh}', fontsize=11, fontweight='bold')
    axes[i+1].axis('off')

plt.tight_layout()
plt.show()

# Use threshold of 30 for subsequent analysis
change_mask = threshold_changes(diff_image, threshold=30)
print(f"Change mask created with threshold=30")
print(f"Changed pixels: {np.sum(change_mask > 0)} ({100*np.sum(change_mask > 0)/change_mask.size:.2f}%)")

## 9. Change Vector Analysis (CVA)

In [ ]:
def change_vector_analysis(before, after, threshold_percentile=90):
    """
    Change Vector Analysis (CVA)
    Computes magnitude and direction of spectral change
    """
    # Normalize images
    before_float = before.astype(np.float32)
    after_float = after.astype(np.float32)
    
    # Compute change vector
    change_vector = after_float - before_float
    
    # Compute magnitude of change
    magnitude = np.sqrt(np.sum(change_vector**2, axis=2))
    
    # Compute direction (angle)
    direction = np.arctan2(change_vector[:, :, 1], change_vector[:, :, 0])
    
    # Threshold by percentile
    threshold_value = np.percentile(magnitude, threshold_percentile)
    change_magnitude_mask = (magnitude > threshold_value).astype(np.uint8) * 255
    
    return magnitude, direction, change_magnitude_mask

# Apply CVA
magnitude, direction, cva_mask = change_vector_analysis(before_proc, after_proc, threshold_percentile=85)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(cv2.cvtColor(before_proc, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Before Image', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(after_proc, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('After Image', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

im1 = axes[1, 0].imshow(magnitude, cmap='jet')
axes[1, 0].set_title('Change Magnitude (CVA)', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')
plt.colorbar(im1, ax=axes[1, 0], fraction=0.046)

axes[1, 1].imshow(cva_mask, cmap='gray')
axes[1, 1].set_title('CVA Change Mask', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"CVA completed successfully!")
print(f"Change magnitude range: [{magnitude.min():.2f}, {magnitude.max():.2f}]")

## 10. Deep Learning Approach - Simple Siamese Network

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def create_simple_change_detection_model(input_shape=(256, 256, 3)):
    """
    Create a simple CNN-based change detection model
    Uses a Siamese-like architecture
    """
    # Input layers
    input_before = layers.Input(shape=input_shape, name='before_image')
    input_after = layers.Input(shape=input_shape, name='after_image')
    
    # Shared feature extractor
    def feature_extractor():
        model = keras.Sequential([
            layers.Conv2D(32, 3, activation='relu', padding='same'),
            layers.MaxPooling2D(2),
            layers.Conv2D(64, 3, activation='relu', padding='same'),
            layers.MaxPooling2D(2),
            layers.Conv2D(128, 3, activation='relu', padding='same'),
            layers.MaxPooling2D(2),
        ])
        return model
    
    extractor = feature_extractor()
    
    # Extract features from both images
    features_before = extractor(input_before)
    features_after = extractor(input_after)
    
    # Concatenate features
    combined = layers.Concatenate()([features_before, features_after])
    
    # Decoder for change detection
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(combined)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    
    # Output change map
    output = layers.Conv2D(1, 1, activation='sigmoid', padding='same', name='change_map')(x)
    
    # Create model
    model = keras.Model(inputs=[input_before, input_after], outputs=output)
    
    return model

# Create model
print("Creating deep learning model...")
model = create_simple_change_detection_model(input_shape=(256, 256, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Model created successfully!")
model.summary()

## 11. Deep Learning Change Detection (Inference)

In [ ]:
# Note: This is inference with an untrained model (for demonstration)
# In practice, you would train this model on labeled change detection data

def predict_changes_dl(model, before, after):
    """
    Predict changes using deep learning model
    """
    # Prepare inputs
    before_input = np.expand_dims(before_norm, axis=0)
    after_input = np.expand_dims(after_norm, axis=0)
    
    # Predict
    change_pred = model.predict([before_input, after_input], verbose=0)
    
    # Remove batch dimension and squeeze
    change_pred = np.squeeze(change_pred)
    
    return change_pred

# Run inference
print("Running deep learning inference...")
dl_change_map = predict_changes_dl(model, before_proc, after_proc)

# Threshold the prediction
dl_change_mask = (dl_change_map > 0.5).astype(np.uint8) * 255

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(cv2.cvtColor(before_proc, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Before Image', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(after_proc, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('After Image', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

im = axes[1, 0].imshow(dl_change_map, cmap='hot', vmin=0, vmax=1)
axes[1, 0].set_title('DL Change Probability', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')
plt.colorbar(im, ax=axes[1, 0], fraction=0.046)

axes[1, 1].imshow(dl_change_mask, cmap='gray')
axes[1, 1].set_title('DL Change Mask', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("Note: This uses an untrained model for demonstration.")

## 12. Visualize Detected Changes with Overlay

In [ ]:
def overlay_changes(original_image, change_mask, color=(255, 0, 0), alpha=0.4):
    """
    Overlay change mask on original image with transparency
    """
    # Create overlay
    overlay = original_image.copy()
    
    # Create red mask for changes
    red_mask = np.zeros_like(original_image)
    red_mask[change_mask > 0] = color
    
    # Blend
    result = cv2.addWeighted(overlay, 1-alpha, red_mask, alpha, 0)
    
    return result

# Create overlays for different methods
overlay_threshold = overlay_changes(after_proc, change_mask, color=(255, 0, 0), alpha=0.5)
overlay_cva = overlay_changes(after_proc, cva_mask, color=(0, 255, 255), alpha=0.5)
overlay_dl = overlay_changes(after_proc, dl_change_mask, color=(0, 255, 0), alpha=0.5)

# Visualize all methods
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

axes[0, 0].imshow(cv2.cvtColor(before_proc, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Before Image', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(after_proc, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('After Image', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

axes[0, 2].imshow(diff_image, cmap='hot')
axes[0, 2].set_title('Difference Map', fontsize=12, fontweight='bold')
axes[0, 2].axis('off')

axes[1, 0].imshow(cv2.cvtColor(overlay_threshold, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Threshold Method (Red)', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(overlay_cva, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('CVA Method (Cyan)', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

axes[1, 2].imshow(cv2.cvtColor(overlay_dl, cv2.COLOR_BGR2RGB))
axes[1, 2].set_title('Deep Learning Method (Green)', fontsize=12, fontweight='bold')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("Change overlays created successfully!")

## 13. Calculate Change Statistics

In [ ]:
def calculate_change_statistics(change_mask, image_shape):
    """
    Calculate statistics about detected changes
    """
    total_pixels = change_mask.size
    changed_pixels = np.sum(change_mask > 0)
    unchanged_pixels = total_pixels - changed_pixels
    
    change_percentage = (changed_pixels / total_pixels) * 100
    
    # Find contours (connected change regions)
    contours, _ = cv2.findContours(change_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    num_change_regions = len(contours)
    
    # Calculate area of largest change
    if contours:
        largest_change_area = max([cv2.contourArea(c) for c in contours])
        average_change_area = changed_pixels / num_change_regions if num_change_regions > 0 else 0
    else:
        largest_change_area = 0
        average_change_area = 0
    
    stats = {
        'total_pixels': total_pixels,
        'changed_pixels': changed_pixels,
        'unchanged_pixels': unchanged_pixels,
        'change_percentage': change_percentage,
        'num_change_regions': num_change_regions,
        'largest_change_area': largest_change_area,
        'average_change_area': average_change_area
    }
    
    return stats

# Calculate statistics for all methods
stats_threshold = calculate_change_statistics(change_mask, before_proc.shape)
stats_cva = calculate_change_statistics(cva_mask, before_proc.shape)
stats_dl = calculate_change_statistics(dl_change_mask, before_proc.shape)

# Display results
print("=" * 70)
print("CHANGE DETECTION STATISTICS")
print("=" * 70)

print("\n📊 THRESHOLD METHOD:")
print(f"  • Total Pixels: {stats_threshold['total_pixels']:,}")
print(f"  • Changed Pixels: {stats_threshold['changed_pixels']:,}")
print(f"  • Change Percentage: {stats_threshold['change_percentage']:.2f}%")
print(f"  • Number of Change Regions: {stats_threshold['num_change_regions']}")
print(f"  • Largest Change Area: {stats_threshold['largest_change_area']:.0f} pixels")

print("\n📊 CHANGE VECTOR ANALYSIS (CVA):")
print(f"  • Total Pixels: {stats_cva['total_pixels']:,}")
print(f"  • Changed Pixels: {stats_cva['changed_pixels']:,}")
print(f"  • Change Percentage: {stats_cva['change_percentage']:.2f}%")
print(f"  • Number of Change Regions: {stats_cva['num_change_regions']}")
print(f"  • Largest Change Area: {stats_cva['largest_change_area']:.0f} pixels")

print("\n📊 DEEP LEARNING METHOD:")
print(f"  • Total Pixels: {stats_dl['total_pixels']:,}")
print(f"  • Changed Pixels: {stats_dl['changed_pixels']:,}")
print(f"  • Change Percentage: {stats_dl['change_percentage']:.2f}%")
print(f"  • Number of Change Regions: {stats_dl['num_change_regions']}")
print(f"  • Largest Change Area: {stats_dl['largest_change_area']:.0f} pixels")

print("\n" + "=" * 70)

## 13.5. Advanced Metrics (Kappa Coefficient, F1 Score, Precision, Recall)

In [ ]:
def calculate_advanced_metrics(predicted_mask, ground_truth_mask):
    """
    Calculate advanced metrics for change detection evaluation
    
    Metrics included:
    - Kappa Coefficient (Cohen's Kappa)
    - F1 Score
    - Precision
    - Recall
    - Overall Accuracy
    - Producer's Accuracy (for both classes)
    - User's Accuracy (for both classes)
    - False Positive Rate
    - False Negative Rate
    """
    # Flatten masks to 1D arrays
    pred = (predicted_mask > 0).flatten().astype(int)
    gt = (ground_truth_mask > 0).flatten().astype(int)
    
    # Calculate confusion matrix elements
    TP = np.sum((pred == 1) & (gt == 1))  # True Positives
    TN = np.sum((pred == 0) & (gt == 0))  # True Negatives
    FP = np.sum((pred == 1) & (gt == 0))  # False Positives
    FN = np.sum((pred == 0) & (gt == 1))  # False Negatives
    
    # Overall Accuracy
    overall_accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    
    # Precision (User's Accuracy for Changed class)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    
    # Recall (Producer's Accuracy for Changed class / Sensitivity)
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    # F1 Score (Harmonic mean of Precision and Recall)
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Kappa Coefficient (Cohen's Kappa)
    po = overall_accuracy  # Observed agreement
    
    # Expected agreement
    total = TP + TN + FP + FN
    pred_positive = TP + FP
    pred_negative = TN + FN
    actual_positive = TP + FN
    actual_negative = TN + FP
    
    pe = ((pred_positive * actual_positive) + (pred_negative * actual_negative)) / (total ** 2) if total > 0 else 0
    
    # Kappa coefficient
    kappa = (po - pe) / (1 - pe) if (1 - pe) != 0 else 0
    
    # Producer's Accuracy (for both classes)
    producer_acc_changed = recall  # Same as recall
    producer_acc_unchanged = TN / (TN + FP) if (TN + FP) > 0 else 0
    
    # User's Accuracy (for both classes)
    user_acc_changed = precision  # Same as precision
    user_acc_unchanged = TN / (TN + FN) if (TN + FN) > 0 else 0
    
    # False Positive Rate (Fall-out)
    fpr = FP / (FP + TN) if (FP + TN) > 0 else 0
    
    # False Negative Rate (Miss rate)
    fnr = FN / (FN + TP) if (FN + TP) > 0 else 0
    
    metrics = {
        'confusion_matrix': {
            'TP': int(TP),
            'TN': int(TN),
            'FP': int(FP),
            'FN': int(FN)
        },
        'kappa_coefficient': kappa,
        'f1_score': f1_score,
        'precision': precision,
        'recall': recall,
        'overall_accuracy': overall_accuracy,
        'producer_accuracy_changed': producer_acc_changed,
        'producer_accuracy_unchanged': producer_acc_unchanged,
        'user_accuracy_changed': user_acc_changed,
        'user_accuracy_unchanged': user_acc_unchanged,
        'false_positive_rate': fpr,
        'false_negative_rate': fnr
    }
    
    return metrics

# Create a synthetic ground truth for demonstration
# In practice, you would have actual ground truth labels
ground_truth = np.zeros_like(change_mask)
ground_truth[100:180, 100:180] = 255  # Simulated true change area
ground_truth[150:200, 150:200] = 255  # Another change area

# Calculate advanced metrics for all methods
metrics_threshold = calculate_advanced_metrics(change_mask, ground_truth)
metrics_cva = calculate_advanced_metrics(cva_mask, ground_truth)
metrics_dl = calculate_advanced_metrics(dl_change_mask, ground_truth)

# Display results
print("=" * 80)
print("ADVANCED CHANGE DETECTION METRICS")
print("=" * 80)
print("\n⚠️  Note: Using synthetic ground truth for demonstration.")
print("    Replace 'ground_truth' with your actual labeled data.\n")

def print_metrics(metrics, method_name):
    print(f"\n📊 {method_name}:")
    print(f"  Confusion Matrix:")
    print(f"    • True Positives (TP):  {metrics['confusion_matrix']['TP']:,}")
    print(f"    • True Negatives (TN):  {metrics['confusion_matrix']['TN']:,}")
    print(f"    • False Positives (FP): {metrics['confusion_matrix']['FP']:,}")
    print(f"    • False Negatives (FN): {metrics['confusion_matrix']['FN']:,}")
    print(f"  ")
    print(f"  Accuracy Metrics:")
    print(f"    • Overall Accuracy:     {metrics['overall_accuracy']:.4f} ({metrics['overall_accuracy']*100:.2f}%)")
    print(f"    • Kappa Coefficient:    {metrics['kappa_coefficient']:.4f}")
    print(f"  ")
    print(f"  Performance Metrics:")
    print(f"    • F1 Score:             {metrics['f1_score']:.4f}")
    print(f"    • Precision:            {metrics['precision']:.4f} ({metrics['precision']*100:.2f}%)")
    print(f"    • Recall (Sensitivity): {metrics['recall']:.4f} ({metrics['recall']*100:.2f}%)")
    print(f"  ")
    print(f"  Error Rates:")
    print(f"    • False Positive Rate:  {metrics['false_positive_rate']:.4f} ({metrics['false_positive_rate']*100:.2f}%)")
    print(f"    • False Negative Rate:  {metrics['false_negative_rate']:.4f} ({metrics['false_negative_rate']*100:.2f}%)")
    print(f"  ")
    print(f"  Class-wise Accuracies:")
    print(f"    • Producer's Acc (Changed):   {metrics['producer_accuracy_changed']:.4f}")
    print(f"    • Producer's Acc (Unchanged): {metrics['producer_accuracy_unchanged']:.4f}")
    print(f"    • User's Acc (Changed):       {metrics['user_accuracy_changed']:.4f}")
    print(f"    • User's Acc (Unchanged):     {metrics['user_accuracy_unchanged']:.4f}")

print_metrics(metrics_threshold, "THRESHOLD METHOD")
print_metrics(metrics_cva, "CHANGE VECTOR ANALYSIS (CVA)")
print_metrics(metrics_dl, "DEEP LEARNING METHOD")

print("\n" + "=" * 80)

# Visualize confusion matrices
fig, axes = plt.subplots(1, 4, figsize=(18, 4))

# Ground truth
axes[0].imshow(ground_truth, cmap='gray')
axes[0].set_title('Ground Truth\n(Synthetic)', fontsize=11, fontweight='bold')
axes[0].axis('off')

# Threshold method
axes[1].imshow(change_mask, cmap='gray')
axes[1].set_title(f'Threshold Method\nKappa: {metrics_threshold["kappa_coefficient"]:.3f} | F1: {metrics_threshold["f1_score"]:.3f}', 
                 fontsize=10, fontweight='bold')
axes[1].axis('off')

# CVA method
axes[2].imshow(cva_mask, cmap='gray')
axes[2].set_title(f'CVA Method\nKappa: {metrics_cva["kappa_coefficient"]:.3f} | F1: {metrics_cva["f1_score"]:.3f}', 
                 fontsize=10, fontweight='bold')
axes[2].axis('off')

# DL method
axes[3].imshow(dl_change_mask, cmap='gray')
axes[3].set_title(f'Deep Learning\nKappa: {metrics_dl["kappa_coefficient"]:.3f} | F1: {metrics_dl["f1_score"]:.3f}', 
                 fontsize=10, fontweight='bold')
axes[3].axis('off')

plt.tight_layout()
plt.show()

## 14. Export Results

In [ ]:
# Create output directory
output_dir = os.path.join(working_dir, 'results')
os.makedirs(output_dir, exist_ok=True)

# Save change masks
cv2.imwrite(os.path.join(output_dir, 'before_image.png'), before_proc)
cv2.imwrite(os.path.join(output_dir, 'after_image.png'), after_proc)
cv2.imwrite(os.path.join(output_dir, 'difference_map.png'), diff_image)
cv2.imwrite(os.path.join(output_dir, 'threshold_change_mask.png'), change_mask)
cv2.imwrite(os.path.join(output_dir, 'cva_change_mask.png'), cva_mask)
cv2.imwrite(os.path.join(output_dir, 'dl_change_mask.png'), dl_change_mask)

# Save overlays
cv2.imwrite(os.path.join(output_dir, 'threshold_overlay.png'), overlay_threshold)
cv2.imwrite(os.path.join(output_dir, 'cva_overlay.png'), overlay_cva)
cv2.imwrite(os.path.join(output_dir, 'dl_overlay.png'), overlay_dl)

# Save statistics to text file
stats_file = os.path.join(output_dir, 'change_statistics.txt')
with open(stats_file, 'w') as f:
    f.write("SATELLITE CHANGE DETECTION RESULTS\n")
    f.write("=" * 80 + "\n\n")
    
    f.write("BASIC STATISTICS:\n")
    f.write("-" * 80 + "\n\n")
    
    f.write("THRESHOLD METHOD:\n")
    for key, value in stats_threshold.items():
        f.write(f"  {key}: {value}\n")
    
    f.write("\nCHANGE VECTOR ANALYSIS (CVA):\n")
    for key, value in stats_cva.items():
        f.write(f"  {key}: {value}\n")
    
    f.write("\nDEEP LEARNING METHOD:\n")
    for key, value in stats_dl.items():
        f.write(f"  {key}: {value}\n")
    
    # Add advanced metrics
    f.write("\n\n" + "=" * 80 + "\n")
    f.write("ADVANCED METRICS (Kappa, F1, Precision, Recall)\n")
    f.write("=" * 80 + "\n\n")
    
    def write_metrics(f, metrics, method_name):
        f.write(f"{method_name}:\n")
        f.write(f"  Confusion Matrix:\n")
        f.write(f"    TP: {metrics['confusion_matrix']['TP']:,}\n")
        f.write(f"    TN: {metrics['confusion_matrix']['TN']:,}\n")
        f.write(f"    FP: {metrics['confusion_matrix']['FP']:,}\n")
        f.write(f"    FN: {metrics['confusion_matrix']['FN']:,}\n")
        f.write(f"  Overall Accuracy: {metrics['overall_accuracy']:.4f}\n")
        f.write(f"  Kappa Coefficient: {metrics['kappa_coefficient']:.4f}\n")
        f.write(f"  F1 Score: {metrics['f1_score']:.4f}\n")
        f.write(f"  Precision: {metrics['precision']:.4f}\n")
        f.write(f"  Recall: {metrics['recall']:.4f}\n")
        f.write(f"  False Positive Rate: {metrics['false_positive_rate']:.4f}\n")
        f.write(f"  False Negative Rate: {metrics['false_negative_rate']:.4f}\n\n")
    
    write_metrics(f, metrics_threshold, "THRESHOLD METHOD")
    write_metrics(f, metrics_cva, "CHANGE VECTOR ANALYSIS (CVA)")
    write_metrics(f, metrics_dl, "DEEP LEARNING METHOD")

# Save ground truth
cv2.imwrite(os.path.join(output_dir, 'ground_truth.png'), ground_truth)

print(f"\nSaved files:")
print("  • before_image.png")
print("  • after_image.png")
print("  • difference_map.png")
print("  • ground_truth.png")
print("  • threshold_change_mask.png")
print("  • cva_change_mask.png")
print("  • dl_change_mask.png")
print("  • threshold_overlay.png")
print("  • cva_overlay.png")
print("  • dl_overlay.png")
print("  • change_statistics.txt (includes advanced metrics)")

## 🚀 How to Use This Notebook

### For Your Own Satellite Images:

1. **Replace the sample image creation** in Section 4 with your own image loading code:
```python
# Load your own satellite images
before_img = cv2.imread('path/to/your/before_image.tif')
after_img = cv2.imread('path/to/your/after_image.tif')
```

2. **For GeoTIFF files**, use rasterio:
```python
import rasterio
with rasterio.open('your_satellite_image.tif') as src:
    image = src.read()  # Read all bands
    image = np.transpose(image, (1, 2, 0))  # Rearrange to (H, W, C)
```

3. **Adjust parameters** based on your data:
   - Threshold values in Section 8
   - CVA percentile in Section 9
   - Model architecture in Section 10 (if training)

### Training the Deep Learning Model:

To train the model on your own labeled data:
```python
# Prepare your training data
X_train_before = [...]  # List of before images
X_train_after = [...]   # List of after images
y_train = [...]         # List of change masks (ground truth)

# Train the model
model.fit(
    [X_train_before, X_train_after],
    y_train,
    epochs=50,
    batch_size=8,
    validation_split=0.2
)
```

### Key Features:
✅ Multiple change detection methods  
✅ Google Colab compatible  
✅ Visualization and statistics  
✅ Export results in multiple formats  
✅ Easy to customize for your data